# データ取得

本章は、DataFrameから列、行、或いは単一の値を取得する方法について説明します。

In [32]:
import polars as pl
from helper.jupyter import row

df = pl.DataFrame(
    {
        "a": [3, 3, 3, 4],
        "b": [4, 12, 6, 7],
        "g": ['A', 'B', 'A', 'B']
    }
)

## 列

PolarsでDataFrameから列データをSeriesとして取得する方法はいくつかあります。

* `DataFrame.to_series()`: インデックスで列を取得します。
* `DataFrame.get_column()`: 列名で列を取得します。
* `DataFrame.get_columns()`: すべての列を取得します。
* `DataFrame.iter_columns()`: 列のイテレーターを取得します。

`DataFrame.to_series()` メソッドを使用すると、指定したインデックスに基づいて列を Series として取得できます。

In [10]:
df.to_series(0)

a
i64
3
3
3
4


`DataFrame.get_column()` メソッドを使用すると、列名を指定して Series を取得できます。`DataFrame["column_name"]`のような辞書形式で列名を指定して Seriesを取得することもできます。

In [14]:
row(df.get_column('b'), df['g'])

b
i64
4
12
6
7
g
str
"""A"""
"""B"""
"""A"""


`DataFrame.get_columns()` メソッドは、DataFrame 内のすべての列を Series のリストとして取得します。

In [12]:
a, b, g = df.get_columns()
a

a
i64
3
3
3
4


`DataFrame.iter_columns()`は、DataFrame内のすべての列を一つずつ返します。

In [16]:
for col in df.iter_columns():
    print(col.name, col.to_list())

a [3, 3, 3, 4]
b [4, 12, 6, 7]
g ['A', 'B', 'A', 'B']


Seriesオブジェクトの`to_numpy()`または`to_list()`メソッドを使用すると、NumPyの配列やリストに変換することができます。

In [7]:
a.to_numpy()

array([3, 3, 3, 4], dtype=int64)

In [8]:
a.to_list()

[3, 3, 3, 4]

## 行

- **`DataFrame.row(index)`**: 特定のインデックスにある行をタプルとして取得します。
- **`DataFrame.rows()`**: DataFrame のすべての行をタプルのリストとして取得します。
- **`DataFrame.iter_rows()`**: DataFrame の各行に対してイテレーションを行い、各行をタプルとして返します。
- **`DataFrame.rows_by_key()`**: 指定したキーに基づいて行をグループ化し、各グループに属する行をタプルのリストとして返します。

`DataFrame.row(index)` は、指定したインデックス（行番号）に対応する行を取得するためのメソッドです。このメソッドは、特定の行をタプル形式で返します。

In [17]:
df.row(2)

(3, 6, 'A')

`named`引数を`True`にすることで、列名を含む辞書形式で返します。

In [22]:
df.row(2, named=True)

{'a': 3, 'b': 6, 'g': 'A'}

`by_predicate`で演算式がTrueになる行を取得することができます。複数の行は条件に満たす場合は、エラーになります。

In [23]:
df.row(by_predicate=pl.col.a == pl.col.a.max())

(4, 7, 'B')

`DataFrame.rows()` は、DataFrame 内のすべての行をタプルのリストとして取得するメソッドです。DataFrame 全体のデータを行単位で操作したい場合に便利です。又、`DataFrame.iter_rows()` は各行に対してイテレーションを行うためのメソッドです。

In [24]:
df.rows()

[(3, 4, 'A'), (3, 12, 'B'), (3, 6, 'A'), (4, 7, 'B')]

In [25]:
df.rows(named=True)

[{'a': 3, 'b': 4, 'g': 'A'},
 {'a': 3, 'b': 12, 'g': 'B'},
 {'a': 3, 'b': 6, 'g': 'A'},
 {'a': 4, 'b': 7, 'g': 'B'}]

In [28]:
for row in df.iter_rows():
    print(row)

(3, 4, 'A')
(3, 12, 'B')
(3, 6, 'A')
(4, 7, 'B')


`DataFrame.rows_by_key()` は、指定したキー（列）に基づいて DataFrame 内の行をグループ化し、そのキーに対応する行をタプルのリストとして返すメソッドです。

In [26]:
df.rows_by_key('g')

defaultdict(list, {'A': [(3, 4), (3, 6)], 'B': [(3, 12), (4, 7)]})

In [27]:
df.rows_by_key('g', named=True)

defaultdict(list,
            {'A': [{'a': 3, 'b': 4}, {'a': 3, 'b': 6}],
             'B': [{'a': 3, 'b': 12}, {'a': 4, 'b': 7}]})

## 単一の値

`DataFrame.item()`は、DataFrameから単一の値を取得するためのメソッドです。

In [25]:
df.item(2, 'a')

3

DataFrameに値が一つだけ含まれる場合、引数を省略できます。列の統計値を取得して、それを別の計算に利用する際によく使用されます。

In [31]:
df.select(pl.col.a.mean()).item() / 100

0.0325